# Spectrograms - CNN Test

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import sqlalchemy
from sqlalchemy import create_engine, inspect

import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import time
from datetime import datetime

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

## Import datasets

In [3]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['aval',
 'bval',
 'demographic',
 'diagnosis',
 'gval',
 'habits',
 'rval',
 'spectrogram']

In [4]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

In [15]:
num_images = 204
width_px = 225
height_px = 166
num_channels = 4
input_shape = (num_images, height_px, width_px, num_channels)

In [11]:
rval_df = dataframes['rval_df']
print(rval_df.head())

   voice156  voice142  voice195  voice181  voice022  voice036  voice208  \
0        47        47        47        47        47        47        47   
1        47        47        47        47        47        47        47   
2        43        43        43        43        43        43        43   
3        23        20        20        18        22        21        19   
4        22        19        19        18        21        20        19   

   voice037  voice023  voice180  ...  voice166  voice199  voice012  voice204  \
0        47        47        47  ...        47        47        47        47   
1        47        47        47  ...        47        47        47        47   
2        43        43        43  ...        43        43        44        43   
3        20        18        18  ...        20        24        25        20   
4        20        17        17  ...        20        24        24        19   

   voice205  voice013  voice007  voice198  voice167  voice173  
0   

In [10]:
rgba_combined = pd.concat([
    dataframes['rval_df'],
    dataframes['gval_df'],
    dataframes['bval_df'],
    dataframes['aval_df']
    ], axis = 1
)

rgba_combined

,voice156,voice142,voice195,voice181,voice022,voice036,voice208,voice037,voice023,voice180,...,voice166,voice199,voice012,voice204,voice205,voice013,voice007,voice198,voice167,voice173
0,47,47,47,47,47,47,47,47,47,47,...,255,255,255,255,255,255,255,255,255,255
1,47,47,47,47,47,47,47,47,47,47,...,255,255,255,255,255,255,255,255,255,255
2,43,43,43,43,43,43,43,43,43,43,...,255,255,255,255,255,255,255,255,255,255
3,23,20,20,18,22,21,19,20,18,18,...,255,255,255,255,255,255,255,255,255,255
4,22,19,19,18,21,20,19,20,17,17,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37345,160,218,199,200,200,160,200,199,200,200,...,255,255,255,255,255,255,255,255,255,255
37346,160,245,224,226,226,160,226,225,226,226,...,255,255,255,255,255,255,255,255,255,255
37347,162,244,223,225,225,163,225,224,225,225,...,255,255,255,255,255,255,255,255,255,255
37348,236,250,246,246,247,236,246,247,246,247,...,255,255,255,255,255,255,255,255,255,255


In [12]:
pixel_array = rgba_combined.values
pixel_array

array([[ 47,  47,  47, ..., 255, 255, 255],
       [ 47,  47,  47, ..., 255, 255, 255],
       [ 43,  43,  43, ..., 255, 255, 255],
       ...,
       [162, 244, 223, ..., 255, 255, 255],
       [236, 250, 246, ..., 255, 255, 255],
       [248, 251, 250, ..., 255, 255, 255]])

In [16]:
height, width = rgba_combined.shape
pixel_array = pixel_array.reshape(input_shape)

pixel_array

array([[[[ 47,  47,  47,  47],
         [ 47,  47,  47,  47],
         [ 47,  47,  47,  47],
         ...,
         [ 47,  47,  47,  47],
         [ 47,  47,  47,  47],
         [ 47,  46,  47,  47]],

        [[ 47,  47,  47,  47],
         [ 47,  47,  46,  46],
         [ 47,  47,  47,  47],
         ...,
         [ 44,  43,  43,  43],
         [ 45,  43,  42,  43],
         [ 43,  43,  43,  43]],

        [[ 43,  42,  42,  43],
         [ 43,  43,  43,  43],
         [ 43,  43,  43,  42],
         ...,
         [ 13,  12,  11,  12],
         [ 13,  14,  14,  13],
         [ 12,  13,  13,  12]],

        ...,

        [[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255]],

        [[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         